# Quick start

In [13]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [14]:
# Download data from datasets
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [15]:
# Create data loaders.
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [16]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [17]:
from torch.nn.modules.linear import Linear
from torch.nn.modules.activation import ReLU, Sigmoid

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [18]:
# Loss Function
loss_fn = nn.CrossEntropyLoss()

# Optimizer
# optimizer = torch.optim.SGD(model.parameters(), lr = 1e-4)
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-4)

In [19]:
# Train
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()

    # batch(0-937), batch_size = 64 => batch * batch_size = 938 * 64 = 60032 > 60000
    for batch, (X, y) in enumerate(dataloader): # Transform dataloader into a enumerate
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [20]:
# Test
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [21]:
epoch = 5
for t in range(epoch):
  print(f"Epoch {t+1}\n-------------------------------")
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)
print("DONE")

Epoch 1
-------------------------------
loss: 2.302735  [    0/60000]
loss: 1.024533  [ 6400/60000]
loss: 0.565655  [12800/60000]
loss: 0.734523  [19200/60000]
loss: 0.613135  [25600/60000]
loss: 0.507171  [32000/60000]
loss: 0.503685  [38400/60000]
loss: 0.685307  [44800/60000]
loss: 0.610994  [51200/60000]
loss: 0.490479  [57600/60000]
Test Error: 
 Accuracy: 82.3%, Avg loss: 0.494783 

Epoch 2
-------------------------------
loss: 0.364641  [    0/60000]
loss: 0.501033  [ 6400/60000]
loss: 0.343851  [12800/60000]
loss: 0.521218  [19200/60000]
loss: 0.456465  [25600/60000]
loss: 0.411296  [32000/60000]
loss: 0.386191  [38400/60000]
loss: 0.621214  [44800/60000]
loss: 0.554179  [51200/60000]
loss: 0.428047  [57600/60000]
Test Error: 
 Accuracy: 84.6%, Avg loss: 0.438098 

Epoch 3
-------------------------------
loss: 0.276228  [    0/60000]
loss: 0.424009  [ 6400/60000]
loss: 0.295987  [12800/60000]
loss: 0.457454  [19200/60000]
loss: 0.373482  [25600/60000]
loss: 0.384365  [32000/600

In [22]:
# Saving Models
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [23]:
# Loading Models
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [28]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

# Put the model into evaluation mode
model.eval()
x, y = test_data[6][0], test_data[6][1]
with torch.no_grad(): # torch.no_grad() means the parameter would not update when evaluation
  pred = model(x)
  predicted, actual = classes[pred[0].argmax(0)], classes[y]
  print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Coat", Actual: "Coat"
